<!-- TOC -->
# Table of Contents

- [🔧 Environment Setup](#🔧-environment-setup)
- [📐 Batch Processing Configuration](#📐-batch-processing-configuration)
- [💾 Initialize Checkpoint System](#💾-initialize-checkpoint-system)
- [📂 Scan Video Directories](#📂-scan-video-directories)
- [🎯 Find Target Videos](#🎯-find-target-videos)
- [💾 Save Video Manifest](#💾-save-video-manifest)
- [🎬 Preview Extraction Configuration](#🎬-preview-extraction-configuration)
- [🎥 Extract Preview Frames](#🎥-extract-preview-frames)
- [📊 Display Frame Previews](#📊-display-frame-previews)
- [📈 Quality Analysis & Recommendations](#📈-quality-analysis-&-recommendations)
- [📚 Quality Metrics Reference](#📚-quality-metrics-reference)
  - [Brightness (Luminance)](#brightness-(luminance))
  - [Blur Score (Laplacian Variance)](#blur-score-(laplacian-variance))
  - [Quartile-Based Outlier Detection](#quartile-based-outlier-detection)
  - [Color Space Conversion (BGR to RGB)](#color-space-conversion-(bgr-to-rgb))
- [💾 Export Selection for Individual Processing](#💾-export-selection-for-individual-processing)
- [📋 Batch Processing Summary](#📋-batch-processing-summary)

<!-- /TOC -->


## 🔧 Environment Setup

This cell establishes the batch preprocessing environment by:

1. **Importing Required Libraries**
  - OpenCV (cv2) for video processing and frame extraction
  - NumPy for array operations
  - Pandas for organizing metadata and results
  - Pathlib for cross-platform file path handling
  - JSON for checkpoint persistence
  - Datetime for timestamp parsing and filtering
  - Logging for process tracking

2. **Setting System Paths**
  - Adding mlops_ops modules to Python path
  - Verifying access to preprocessing utilities

3. **Initializing Checkpoint System**
  - Loading any previous processing state
  - Setting up progress tracking variables
  - Establishing failure recovery mechanism

**Note**: Run this cell first to ensure all dependencies are available before proceeding with batch processing.

In [ ]:
# Cell 2 - Environment Setup
import numpy as np
import pandas as pd
from pathlib import Path
import json
from datetime import datetime, timedelta
import logging
import os
import sys

# Add mlops modules to path
sys.path.insert(0, '../lib')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Check for OpenCV
try:
    import cv2
    print(f"✓ OpenCV version: {cv2.__version__}")
except ImportError:
    print("⚠️ OpenCV not installed. Install with: pip install opencv-python")
    print("   Continuing without video processing capabilities...")
    cv2 = None

print(f"✓ Python version: {sys.version.split()[0]}")
print(f"✓ Working directory: {os.getcwd()}")

## 📐 Batch Processing Configuration

Define the core parameters for daily batch preprocessing:

- **Target Time**: Extract frames from videos closest to 12:00 PM EST
- **Date Filter**: Process only videos from yesterday (full calendar day)
- **Frame Count**: Number of frames to extract per video
- **Input Path**: Base directory containing camera subdirectories
- **Output Path**: Where to save extracted frames
- **File Pattern**: Expected video filename format (CAMERA_YYYYMMDD_HHMMSS.mp4)

This configuration serves as the single source of truth for the batch processing workflow.

In [ ]:
# Cell 3 - Batch Processing Configuration
from datetime import datetime, timedelta

# Configuration
CONFIG = {
   # Time targeting
   'TARGET_TIME': '120000',  # 12:00:00 in 24-hour format
   'TARGET_HOUR': 12,
   
   # Date filtering - yesterday only
   'PROCESS_DATE': (datetime.now() - timedelta(days=1)).strftime('%Y%m%d'),
   
   # Frame extraction
   'FRAMES_PER_VIDEO': 10,
   
   # Paths
   'INPUT_DIR': Path.home() / 'traffic-recordings',
   'OUTPUT_DIR': Path('batch_processed_frames'),
   
   # File pattern
   'VIDEO_PATTERN': '*_{date}_*.mp4',  # Will be formatted with PROCESS_DATE
   'FILENAME_FORMAT': '{camera}_{date}_{time}.mp4'  # Expected format
}

# Display configuration
print("Batch Processing Configuration:")
print(f"  Target Date: {CONFIG['PROCESS_DATE']}")
print(f"  Target Time: {CONFIG['TARGET_TIME']} (12:00:00)")
print(f"  Frames per video: {CONFIG['FRAMES_PER_VIDEO']}")
print(f"  Input: {CONFIG['INPUT_DIR']}")
print(f"  Output: {CONFIG['OUTPUT_DIR']}")

## 💾 Initialize Checkpoint System

Create checkpoint functionality to track processing progress and enable recovery from interruptions. This system saves state after each video completes, allowing the workflow to resume from the last successful video if stopped.

In [ ]:
# Cell 4 - Initialize Checkpoint System
CHECKPOINT_FILE = "batch_preprocessing_checkpoint.json"
start_time = datetime.now()

def load_checkpoint():
   """Load previous progress if exists"""
   if Path(CHECKPOINT_FILE).exists():
       with open(CHECKPOINT_FILE, 'r') as f:
           checkpoint = json.load(f)
           print(f"✓ Loaded checkpoint: {len(checkpoint['processed'])} videos already processed")
           return checkpoint
   return {
       "processed": [], 
       "failed": [], 
       "last_completed": None,
       "process_date": CONFIG['PROCESS_DATE'],
       "start_time": start_time.isoformat()
   }

def save_checkpoint(checkpoint):
   """Save current progress"""
   checkpoint['last_updated'] = datetime.now().isoformat()
   with open(CHECKPOINT_FILE, 'w') as f:
       json.dump(checkpoint, f, indent=2)

# Initialize checkpoint
checkpoint = load_checkpoint()

# Verify checkpoint is for current date
if checkpoint.get('process_date') != CONFIG['PROCESS_DATE']:
   print(f"⚠️  Checkpoint is from {checkpoint.get('process_date')}, starting fresh for {CONFIG['PROCESS_DATE']}")
   checkpoint = {
       "processed": [], 
       "failed": [], 
       "last_completed": None,
       "process_date": CONFIG['PROCESS_DATE'],
       "start_time": start_time.isoformat()
   }

print(f"✓ Checkpoint system ready")

## 📂 Scan Video Directories

Enumerate all camera subdirectories and count available videos from yesterday's date. This provides an overview of the data available for processing and identifies any cameras that may be missing recordings.

In [ ]:
# Cell 5 - Scan Video Directories (Updated)
camera_dirs = sorted([d for d in CONFIG['INPUT_DIR'].iterdir() if d.is_dir() and d.name.startswith('ATL-')])
print(f"Found {len(camera_dirs)} camera directories\n")

# Count videos per camera for yesterday
video_counts = {}
date_folder = CONFIG['PROCESS_DATE'][:4] + '-' + CONFIG['PROCESS_DATE'][4:6] + '-' + CONFIG['PROCESS_DATE'][6:8]  # Convert to YYYY-MM-DD
pattern = CONFIG['VIDEO_PATTERN'].format(date=CONFIG['PROCESS_DATE'])

for cam_dir in camera_dirs:
    date_dir = cam_dir / date_folder
    if date_dir.exists():
        videos = list(date_dir.glob(pattern))
        video_counts[cam_dir.name] = len(videos)
        
        if len(videos) == 0:
            print(f"⚠️  {cam_dir.name}: No videos in {date_folder}")
        else:
            print(f"✓ {cam_dir.name}: {len(videos)} videos")
    else:
        video_counts[cam_dir.name] = 0
        print(f"⚠️  {cam_dir.name}: No {date_folder} directory")

total_videos = sum(video_counts.values())
print(f"\nTotal videos available: {total_videos}")
print(f"Cameras with recordings: {sum(1 for v in video_counts.values() if v > 0)}/{len(camera_dirs)}")

## 🎯 Find Target Videos

Parse timestamps from video filenames and identify the video closest to 12:00:00 (noon) for each camera. This creates the final list of videos to process.

In [ ]:
# Cell 6 - Find Target Videos
from datetime import datetime

def parse_timestamp(filename):
   """Extract timestamp from filename and calculate minutes from midnight"""
   # Format: ATL-XXXX_YYYYMMDD_HHMMSS.mp4
   parts = filename.stem.split('_')
   if len(parts) >= 3:
       time_str = parts[2]
       hours = int(time_str[:2])
       minutes = int(time_str[2:4])
       seconds = int(time_str[4:6])
       return hours * 60 + minutes  # Minutes from midnight
   return None

def find_closest_to_noon(video_list):
   """Find video closest to 12:00:00"""
   target_minutes = CONFIG['TARGET_HOUR'] * 60  # 720 minutes (12:00)
   
   closest_video = None
   min_diff = float('inf')
   
   for video in video_list:
       minutes = parse_timestamp(video)
       if minutes is not None:
           diff = abs(minutes - target_minutes)
           if diff < min_diff:
               min_diff = diff
               closest_video = video
   
   return closest_video, min_diff

# Find target videos for each camera
target_videos = []
date_folder = CONFIG['PROCESS_DATE'][:4] + '-' + CONFIG['PROCESS_DATE'][4:6] + '-' + CONFIG['PROCESS_DATE'][6:8]

for cam_dir in camera_dirs:
   date_dir = cam_dir / date_folder
   if date_dir.exists():
       videos = list(date_dir.glob(f"{cam_dir.name}_*.mp4"))
       if videos:
           closest, diff_minutes = find_closest_to_noon(videos)
           if closest:
               target_videos.append({
                   'camera': cam_dir.name,
                   'video_path': closest,
                   'time_diff_minutes': diff_minutes
               })
               time_str = closest.stem.split('_')[2]
               print(f"{cam_dir.name}: {time_str[:2]}:{time_str[2:4]}:{time_str[4:6]} ({diff_minutes} min from noon)")

print(f"\nTotal videos to process: {len(target_videos)}")

## 💾 Save Video Manifest

Create a manifest file containing all selected videos (one per camera closest to noon) with metadata. This manifest serves as the input for individual video preprocessing and review.

In [ ]:
# Cell 7 - Save Video Manifest
manifest_data = {
   'processing_date': CONFIG['PROCESS_DATE'],
   'target_time': CONFIG['TARGET_TIME'],
   'created_at': datetime.now().isoformat(),
   'total_cameras': len(camera_dirs),
   'videos_found': len(target_videos),
   'videos': []
}

for video_info in target_videos:
   video_path = video_info['video_path']
   time_str = video_path.stem.split('_')[2]
   
   manifest_data['videos'].append({
       'camera': video_info['camera'],
       'filename': video_path.name,
       'full_path': str(video_path),
       'recording_time': f"{time_str[:2]}:{time_str[2:4]}:{time_str[4:6]}",
       'time_diff_minutes': video_info['time_diff_minutes'],
       'file_size_mb': round(video_path.stat().st_size / (1024*1024), 2)
   })

# Save manifest
manifest_file = Path(f"batch_manifest_{CONFIG['PROCESS_DATE']}.json")
with open(manifest_file, 'w') as f:
   json.dump(manifest_data, f, indent=2)

# Also save as CSV for easy viewing
df_manifest = pd.DataFrame(manifest_data['videos'])
csv_file = Path(f"batch_manifest_{CONFIG['PROCESS_DATE']}.csv")
df_manifest.to_csv(csv_file, index=False)

print(f"✓ Saved manifest: {manifest_file}")
print(f"✓ Saved CSV: {csv_file}")
print(f"\nSummary:")
print(f"  Videos selected: {len(target_videos)}")
print(f"  Total size: {df_manifest['file_size_mb'].sum():.1f} MB")
print(f"  Average time from noon: {df_manifest['time_diff_minutes'].mean():.1f} minutes")

## 🎬 Preview Extraction Configuration

Define parameters for extracting sample frames from each video. These settings control how many frames to extract and from which portion of the video to generate quality previews.

In [ ]:
# Cell 8 - Preview Extraction Configuration
PREVIEW_CONFIG = {
   'frames_per_video': 5,          # Number of frames to extract per video
   'extraction_duration': 60,      # Extract from first 60 seconds only
   'preview_dir': Path('batch_preview_frames'),
   'max_videos_to_preview': 10     # Limit for initial testing
}

# Create output directory
PREVIEW_CONFIG['preview_dir'].mkdir(exist_ok=True)

print("Preview Configuration:")
print(f"  Frames per video: {PREVIEW_CONFIG['frames_per_video']}")
print(f"  Duration to sample: {PREVIEW_CONFIG['extraction_duration']}s")
print(f"  Output directory: {PREVIEW_CONFIG['preview_dir']}")
print(f"  Max videos: {PREVIEW_CONFIG['max_videos_to_preview']}")

## 🎥 Extract Preview Frames

Process selected videos to extract sample frames with quality metrics. This generates visual previews and calculates brightness/blur scores to help identify videos requiring individual review.

In [ ]:
# Install OpenCV
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python"])

In [ ]:
# Cell 9 - Extract Preview Frames
import cv2

def extract_preview_frames(video_path, output_dir, num_frames=5, duration_seconds=60):
   """Extract evenly spaced frames from video for preview"""
   cap = cv2.VideoCapture(str(video_path))
   if not cap.isOpened():
       return None
   
   fps = cap.get(cv2.CAP_PROP_FPS)
   total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
   duration_frames = min(int(fps * duration_seconds), total_frames)
   
   # Calculate frame indices
   indices = np.linspace(0, duration_frames-1, num_frames, dtype=int)
   
   frames_data = []
   for idx in indices:
       cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
       ret, frame = cap.read()
       if ret:
           # Calculate metrics
           gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
           brightness = np.mean(gray)
           blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
           
           # Save frame
           frame_filename = f"frame_{idx:04d}.jpg"
           frame_path = output_dir / frame_filename
           cv2.imwrite(str(frame_path), frame)
           
           frames_data.append({
               'index': idx,
               'brightness': brightness,
               'blur_score': blur_score,
               'path': frame_path
           })
   
   cap.release()
   return frames_data

# Process videos
print("Extracting preview frames...")
preview_results = []

for i, video_info in enumerate(target_videos[:PREVIEW_CONFIG['max_videos_to_preview']]):
   camera = video_info['camera']
   video_path = video_info['video_path']
   
   # Create output directory
   output_dir = PREVIEW_CONFIG['preview_dir'] / f"{camera}_{CONFIG['PROCESS_DATE']}"
   output_dir.mkdir(exist_ok=True)
   
   # Extract frames
   frames_data = extract_preview_frames(
       video_path, 
       output_dir,
       PREVIEW_CONFIG['frames_per_video'],
       PREVIEW_CONFIG['extraction_duration']
   )
   
   if frames_data:
       avg_brightness = np.mean([f['brightness'] for f in frames_data])
       avg_blur = np.mean([f['blur_score'] for f in frames_data])
       
       preview_results.append({
           'camera': camera,
           'video_path': video_path,
           'frames_extracted': len(frames_data),
           'avg_brightness': avg_brightness,
           'avg_blur_score': avg_blur,
           'frames_data': frames_data
       })
       
       print(f"✓ {camera}: {len(frames_data)} frames, brightness={avg_brightness:.1f}, blur={avg_blur:.1f}")
   else:
       print(f"✗ {camera}: Failed to extract frames")

print(f"\nCompleted preview extraction for {len(preview_results)} videos")

## 📊 Display Frame Previews

Generate visual grid showing extracted frames from each camera with quality metrics. This provides a quick overview to identify which videos need closer inspection.

In [ ]:
# Cell 10 - Display Frame Previews
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(preview_results), PREVIEW_CONFIG['frames_per_video'], 
                       figsize=(PREVIEW_CONFIG['frames_per_video'] * 3, len(preview_results) * 2.5))

if len(preview_results) == 1:
   axes = axes.reshape(1, -1)

for cam_idx, result in enumerate(preview_results):
   camera = result['camera']
   
   # Display each frame
   for frame_idx, frame_data in enumerate(result['frames_data']):
       ax = axes[cam_idx, frame_idx]
       
       # Read and display frame
       img = cv2.imread(str(frame_data['path']))
       img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
       ax.imshow(img_rgb)
       ax.set_title(f"Frame {frame_data['index']}\nB:{frame_data['brightness']:.0f} S:{frame_data['blur_score']:.0f}", 
                   fontsize=8)
       ax.axis('off')
   
   # Add camera label
   axes[cam_idx, 0].text(-0.1, 0.5, f"{camera}\nAvg B:{result['avg_brightness']:.0f}\nAvg S:{result['avg_blur_score']:.0f}", 
                         transform=axes[cam_idx, 0].transAxes,
                         ha='right', va='center', fontsize=10, weight='bold')

plt.suptitle(f'Batch Preview - {CONFIG["PROCESS_DATE"]} @ 12:00', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

# Summary statistics
df_preview = pd.DataFrame([{
   'camera': r['camera'],
   'brightness': r['avg_brightness'],
   'blur_score': r['avg_blur_score']
} for r in preview_results])

print(f"\nQuality Summary:")
print(f"  Brightness range: {df_preview['brightness'].min():.1f} - {df_preview['brightness'].max():.1f}")
print(f"  Blur score range: {df_preview['blur_score'].min():.1f} - {df_preview['blur_score'].max():.1f}")

## 📈 Quality Analysis & Recommendations

Analyze the extracted frames to identify patterns and generate recommendations for which videos may need individual review based on quality metrics.

In [ ]:
# Cell 11 - Quality Analysis & Recommendations
# Analyze quality metrics
df_quality = pd.DataFrame([{
   'camera': r['camera'],
   'brightness': r['avg_brightness'],
   'blur_score': r['avg_blur_score']
} for r in preview_results])

# Define quality thresholds
brightness_low = df_quality['brightness'].quantile(0.25)
brightness_high = df_quality['brightness'].quantile(0.75)
blur_low = df_quality['blur_score'].quantile(0.25)
blur_high = df_quality['blur_score'].quantile(0.75)

# Identify videos needing review
needs_review = []

for r in preview_results:
   issues = []
   if r['avg_brightness'] < brightness_low:
       issues.append('low brightness')
   elif r['avg_brightness'] > brightness_high:
       issues.append('high brightness')
   
   if r['avg_blur_score'] < blur_low:
       issues.append('high blur')
   
   if issues:
       needs_review.append({
           'camera': r['camera'],
           'issues': ', '.join(issues),
           'brightness': r['avg_brightness'],
           'blur_score': r['avg_blur_score']
       })

# Display recommendations
print("Quality Analysis Results:")
print(f"  Brightness quartiles: Q1={brightness_low:.1f}, Q3={brightness_high:.1f}")
print(f"  Blur score quartiles: Q1={blur_low:.1f}, Q3={blur_high:.1f}")

if needs_review:
   print(f"\nVideos needing review ({len(needs_review)}):")
   for video in needs_review:
       print(f"  {video['camera']}: {video['issues']}")
else:
   print("\nAll videos within normal quality ranges")

# Create scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df_quality['brightness'], df_quality['blur_score'], s=100)

for idx, row in df_quality.iterrows():
   plt.annotate(row['camera'], (row['brightness'], row['blur_score']), 
               xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.axvline(brightness_low, color='red', linestyle='--', alpha=0.5, label='Brightness Q1')
plt.axvline(brightness_high, color='red', linestyle='--', alpha=0.5, label='Brightness Q3')
plt.axhline(blur_low, color='blue', linestyle='--', alpha=0.5, label='Blur Q1')

plt.xlabel('Brightness')
plt.ylabel('Blur Score (higher = sharper)')
plt.title(f'Video Quality Distribution - {CONFIG["PROCESS_DATE"]}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 📚 Quality Metrics Reference

### Brightness (Luminance)
Average pixel intensity across the image, measured on a 0-255 scale for 8-bit images.
- **Calculation**: Mean of grayscale pixel values
- **Reference**: [OpenCV Image Processing](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)
- **MLOps Context**: [Google Cloud - Image Quality Assessment](https://cloud.google.com/vision/docs/detecting-properties)

### Blur Score (Laplacian Variance)
Measures image sharpness by computing variance of the Laplacian operator output.
- **Higher values** = Sharper image (more edge detail)
- **Lower values** = Blurrier image (less edge detail)
- **Technical Details**: [Laplacian Operator - SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.laplace.html)
- **Research Paper**: [Diatom autofocusing in brightfield microscopy](https://www.researchgate.net/publication/234073097_Diatom_autofocusing_in_brightfield_microscopy_A_comparative_study)

### Quartile-Based Outlier Detection
Statistical method using Q1 (25th percentile) and Q3 (75th percentile) to identify anomalies.
- **Pandas Documentation**: [DataFrame.quantile](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html)
- **Statistical Background**: [NIST - Quartiles](https://www.itl.nist.gov/div898/handbook/prc/section2/prc252.htm)

### Color Space Conversion (BGR to RGB)
OpenCV uses BGR format by default; conversion needed for matplotlib display.
- **OpenCV Reference**: [Color Space Conversions](https://docs.opencv.org/4.x/d8/d01/group__imgproc__color__conversions.html)
- **Why BGR?**: [Historical reasons from Windows API](https://learnopencv.com/why-does-opencv-use-bgr-color-format/)

## 💾 Export Selection for Individual Processing

Create a queue file listing which cameras to process individually. This file will be read by the individual preprocessing notebook to focus on specific videos that need detailed review.

Selection options:
- **Automatic**: Based on quality outliers identified above
- **Manual**: Specify camera IDs directly
- **All**: Process all previewed cameras

In [ ]:
# Cell 12 - Export Selection for Individual Processing
# Allow manual selection or use quality-based recommendations
selected_cameras = []

# Option 1: Auto-select based on quality issues
for video in needs_review:
   selected_cameras.append(video['camera'])

# Option 2: Manual override (uncomment and modify as needed)
# selected_cameras = ['ATL-0006', 'ATL-0027', 'ATL-0069']  

# Option 3: Select all
# selected_cameras = [r['camera'] for r in preview_results]

# Save selection
selection_data = {
   'batch_date': CONFIG['PROCESS_DATE'],
   'selected_cameras': selected_cameras,
   'selection_criteria': 'quality_based',  # or 'manual' or 'all'
   'created_at': datetime.now().isoformat()
}

selection_file = Path(f"individual_processing_queue_{CONFIG['PROCESS_DATE']}.json")
with open(selection_file, 'w') as f:
   json.dump(selection_data, f, indent=2)

print(f"✓ Saved {len(selected_cameras)} cameras for individual processing")
print(f"  File: {selection_file}")
if selected_cameras:
   print(f"  Cameras: {', '.join(selected_cameras)}")

## 📋 Batch Processing Summary

Generate final summary report showing what was accomplished and next steps for the workflow.

In [ ]:
# Cell 13 - Batch Processing Summary
print("="*60)
print(f"BATCH PROCESSING SUMMARY - {CONFIG['PROCESS_DATE']}")
print("="*60)

print(f"\n📊 Processing Statistics:")
print(f"  Total cameras: {len(camera_dirs)}")
print(f"  Videos found: {len(target_videos)}")
print(f"  Videos previewed: {len(preview_results)}")
print(f"  Frames extracted: {len(preview_results) * PREVIEW_CONFIG['frames_per_video']}")

print(f"\n📈 Quality Overview:")
print(f"  Avg brightness: {df_quality['brightness'].mean():.1f} (range: {df_quality['brightness'].min():.1f}-{df_quality['brightness'].max():.1f})")
print(f"  Avg blur score: {df_quality['blur_score'].mean():.0f} (range: {df_quality['blur_score'].min():.0f}-{df_quality['blur_score'].max():.0f})")
print(f"  Videos flagged for review: {len(needs_review)}")

print(f"\n📁 Output Files:")
print(f"  Manifest: batch_manifest_{CONFIG['PROCESS_DATE']}.json")
print(f"  Preview frames: {PREVIEW_CONFIG['preview_dir']}/")
print(f"  Individual queue: {selection_file.name}")

print(f"\n✅ Next Steps:")
print(f"  1. Review visual previews above")
print(f"  2. Run individual preprocessing notebook")
print(f"  3. Load queue file: {selection_file.name}")

print(f"\nCompleted at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")